<a href="https://colab.research.google.com/github/Samgomes2510/dashborad-ecommerce/blob/main/Otimizacao_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importação de bibliotecas necessárias
from pyspark.sql import SparkSession

# Criação da SparkSession (se não tiver sido criada ainda)
spark = SparkSession.builder.appName('OtimizacaoProjeto').getOrCreate()

# 1️⃣ Leitura do arquivo 'videos-preparados.snappy.parquet'
df_video = spark.read.parquet('videos-preparados.snappy.parquet')

# 2️⃣ Leitura do arquivo 'video-comments-tratados.snappy.parquet'
df_comments = spark.read.parquet('video-comments-tratados.snappy.parquet')

# 3️⃣ Criação de tabelas temporárias para SQL
df_video.createOrReplaceTempView('videos')
df_comments.createOrReplaceTempView('comments')

# 4️⃣ JOIN usando spark.sql
join_video_comments = spark.sql("""
    SELECT v.*, c.Comment, c.Sentiment
    FROM videos v
    INNER JOIN comments c
    ON v.Video ID = c.Video ID
""")

# Visualização simples para garantir que o join está certo
join_video_comments.show(5)



# 5️⃣ Reparticionar para otimizar a distribuição antes do join (por exemplo, usando 8 partitions)
df_video_repart = df_video.repartition(8, 'Video ID')
df_comments_repart = df_comments.repartition(8, 'Video ID')

# Criando novas tabelas temporárias com os repartitionados
df_video_repart.createOrReplaceTempView('videos_repart')
df_comments_repart.createOrReplaceTempView('comments_repart')

# JOIN novamente usando spark.sql nas tabelas reparticionadas
join_video_comments_repart = spark.sql("""
    SELECT v.*, c.Comment, c.Sentiment
    FROM videos_repart v
    INNER JOIN comments_repart c
    ON v.Video ID = c.Video ID
""")

# Coalesce para reduzir o número de partições após o join (opcional - ajusta conforme necessário)
join_video_comments_repart = join_video_comments_repart.coalesce(4)

# Visualização simples para checar
join_video_comments_repart.show(5)



#6 Plano de execução do join normal
print("Plano de execução - join normal:")
join_video_comments.explain()

# Plano de execução do join com repartition + coalesce
print("Plano de execução - join com repartition/coalesce:")
join_video_comments_repart.explain()



#7 Selecionando só as colunas necessárias ANTES do join para reduzir dados em memória
df_video_opt = df_video.select('Video ID', 'Title', 'Published At', 'Keyword', 'Likes')
df_comments_opt = df_comments.select('Video ID', 'Comment', 'Sentiment')

# Repartition para otimizar o join (baseado na chave de junção)
df_video_opt = df_video_opt.repartition(8, 'Video ID')
df_comments_opt = df_comments_opt.repartition(8, 'Video ID')

# Criação de tabelas temporárias otimizadas
df_video_opt.createOrReplaceTempView('videos_opt')
df_comments_opt.createOrReplaceTempView('comments_opt')

# JOIN otimizado com menos dados
join_video_comments_opt = spark.sql("""
    SELECT v.Video ID, v.Title, v.Published At, v.Keyword, v.Likes,
           c.Comment, c.Sentiment
    FROM videos_opt v
    INNER JOIN comments_opt c
    ON v.Video ID = c.Video ID
""")

# Visualizar para checar
join_video_comments_opt.show(5)

# Plano de execução otimizado
print("Plano de execução - join otimizado:")
join_video_comments_opt.explain()


#8 Salva o join otimizado no formato parquet
join_video_comments_opt.write.mode('overwrite').parquet('join-videos-comments-otimizado')

In [28]:
!pip install pyspark

In [29]:
import pyspark
import time

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.storagelevel import StorageLevel

In [30]:
# Criação da SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
from google.colab import files
files.upload()

Saving videos-preparados.snappy (1).parquet to videos-preparados.snappy (1).parquet


{'videos-preparados.snappy (1).parquet': b'PAR1\x15\x00\x15\xdc\xc9\x0e\x15\xe8\xc3\n\x15\xa0\xd3\xe5\x9e\n\x1c\x15\x9a\x1d\x15\x00\x15\x06\x15\x08\x00\x00\xee\xa4\x07\xf0C\x03\x00\x00\x00\x9a\x1d\x01e\x00\x00\x00ASMR MUKBANG DOUBLE BIG MAC &amp; CHEESY HASH BROWNS &amp\x01\x19\xf4E\x10ICKEN NUGGETS (No Talking) EATING SOUNDS(\x00\x00\x00Deadly car bomb detonates outside Moscow:\x00\x00\x00How Biden&#39;s student loan forgiveness program will work=\x00\x00\x00Celebrating My 400 Pound Milestone.... McDonald&#39;s Mukbang#\x00\x00\x00Physics Review - Basic IntroductionH\x00\x00\x00Eating ONLY KOREAN GROCERY STORE FOOD for 24 Hours! Lotte Mart FOOD TOUR&\x00\x00\x0019 Year-Old Starts a Trucking BusinessJ\x00\x00\x00SAT\xc2\xae Tips and Trickzzz: Three Tips and Strategies to INCREASE Your Score!W\x00\x00\x00I found SECRET ROAD to OBUNGA PLANET in MINECRAFT animation! THE MAN FROM WINDOW ScoobyU\x00\x00\x00DON&#39;T USE DOMAIN.COM Before Watch THIS VIDEO! Best Domain Registrar  for Business

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from os import path


In [ ]:
# Please provide the code snippet where this error occurs.  I need the code to help fix the error.
# This message indicates a file path issue, and the solution depends on the context.
# For example, if this was printed from within a Python script, you might use:

# import os
# if not os.path.exists("/content/videos_preparados.snapp.parquet"):
#     print("File does not exist")
# else:
#     # Proceed with file operations
#     pass


# Or if you were using pandas:
# import pandas as pd
# try:
#   df = pd.read_parquet("/content/videos_preparados.snapp.parquet")
# except FileNotFoundError:
#   print("File not found")


# Or you might need to download the file or verify the correct path.  Provide more context!

In [34]:
!ls /content

drive  sample_data


In [39]:
# 1️ Leitura do arquivo
df_video = spark.read.parquet('drive/MyDrive/Colab Notebooks/spar/videos-preparados-parquet')

In [40]:
# 2️ Leitura do arquivo
df_comments = spark.read.parquet('drive/MyDrive/Colab Notbooks/spark/videos-comments-tratados-parquet')

In [44]:
# 3️ Criação de tabelas temporárias para SQL
df_video.createOrReplaceTempView('videos')
df_comments.createOrReplaceTempView('comments')

In [45]:
# 4️ JOIN usando spark.sql
join_videos_comments = spark.sql("""
    SELECT v.*, c.Comment, c.Sentiment
    FROM videos v
    INNER JOIN comments c
    ON v.`Video ID` = c.`Video ID`  -- Enclose 'Video ID' in backticks
""")

# Visualização simples para garantir que o join está certo
join_videos_comments.show(5)

+-----------+--------------------+------------+-------+-----+--------+------+-----------+----+--------------------+---------+-------------+-----+-------------+--------------------+--------------------+--------------------+--------------------+---------+
|   Video ID|               Title|Published At|Keyword|Likes|Comments| Views|Interaction|Year|             Comment|Sentiment|Likes Comment|Month|Keyword Index|            Features|     Features Normal|        Features PCA|             Comment|Sentiment|
+-----------+--------------------+------------+-------+-----+--------+------+-----------+----+--------------------+---------+-------------+-----+-------------+--------------------+--------------------+--------------------+--------------------+---------+
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672|135612|     139691|2022|Let's not forget ...|        1|           95|    8|         17.0|[3407.0,135612.0,...|[0.02511243093431...|[0.39522435928489...|Let's not forget

In [57]:
# 5️ Reparticionar para otimizar a distribuição antes do join (por exemplo, usando 8 partitions)
df_videos_repart = df_video.repartition(8, 'Video ID')
df_comments_repart = df_comments.repartition(8, 'Video ID')

# Criando novas tabelas temporárias com os repartitionados
df_videos_repart.createOrReplaceTempView('videos_repart')
df_comments_repart.createOrReplaceTempView('comments_repart')

# JOIN novamente usando spark.sql nas tabelas reparticionadas
join_videos_comments_repart = spark.sql("""
    SELECT v.*, c.Comment, c.Sentiment
    FROM videos_repart v
    INNER JOIN comments_repart c
    ON v.`Video ID` = c.`Video ID`
""")  #Removed the # and the comment after it

# Coalesce para reduzir o número de partições após o join (opcional - ajusta conforme necessário)
join_videos_comments_repart = join_videos_comments_repart.coalesce(4)

# Visualização simples para checar
join_videos_comments_repart.show(5)

+-----------+--------------------+------------+-------+-----+--------+------+-----------+----+--------------------+---------+-------------+-----+-------------+--------------------+--------------------+--------------------+--------------------+---------+
|   Video ID|               Title|Published At|Keyword|Likes|Comments| Views|Interaction|Year|             Comment|Sentiment|Likes Comment|Month|Keyword Index|            Features|     Features Normal|        Features PCA|             Comment|Sentiment|
+-----------+--------------------+------------+-------+-----+--------+------+-----------+----+--------------------+---------+-------------+-----+-------------+--------------------+--------------------+--------------------+--------------------+---------+
|pT_9hntWj34|Cool Tech Under $...|  2022-08-06|   tech|20999|    3091|413179|     437269|2022|I’m always lookin...|        2|          317|    8|         17.0|[20999.0,413179.0...|[0.05075689216362...|[0.4029173527927699]|I’m always looki

In [58]:
#6 Plano de execução do join normal
print("Plano de execução - join normal:")
join_videos_comments.explain()

# Plano de execução do join com repartition + coalesce
print("Plano de execução - join com repartition/coalesce:")
join_videos_comments_repart.explain()


Plano de execução - join normal:
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [Video ID#34, Title#35, Published At#36, Keyword#37, Likes#38, Comments#39, Views#40, Interaction#41, Year#42, Comment#43, Sentiment#44, Likes Comment#45, Month#46, Keyword Index#47, Features#48, Features Normal#49, Features PCA#50, Comment#77, Sentiment#79]
   +- BroadcastHashJoin [Video ID#34], [Video ID#68], Inner, BuildRight, false
      :- Filter isnotnull(Video ID#34)
      :  +- FileScan parquet [Video ID#34,Title#35,Published At#36,Keyword#37,Likes#38,Comments#39,Views#40,Interaction#41,Year#42,Comment#43,Sentiment#44,Likes Comment#45,Month#46,Keyword Index#47,Features#48,Features Normal#49,Features PCA#50] Batched: true, DataFilters: [isnotnull(Video ID#34)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/drive/MyDrive/Colab Notebooks/spar/videos-preparados-par..., PartitionFilters: [], PushedFilters: [IsNotNull(`Video ID`)], ReadSchema: struct<Video ID:stri

In [59]:
#7 Selecionando só as colunas necessárias ANTES do join para reduzir dados em memória
df_videos_opt = df_video.select('Video ID', 'Title', 'Published At', 'Keyword', 'Likes')
df_comments_opt = df_comments.select('Video ID', 'Comment', 'Sentiment')

# Repartition para otimizar o join (baseado na chave de junção)
df_videos_opt = df_videos_opt.repartition(8, 'Video ID')
df_comments_opt = df_comments_opt.repartition(8, 'Video ID')

# Criação de tabelas temporárias otimizadas
df_videos_opt.createOrReplaceTempView('videos_opt')
df_comments_opt.createOrReplaceTempView('comments_opt')

# JOIN otimizado com menos dados
join_videos_comments_opt = spark.sql("""
    SELECT v.`Video ID`, v.Title, v.`Published At`, v.Keyword, v.Likes,
           c.Comment, c.Sentiment
    FROM videos_opt v
    INNER JOIN comments_opt c
    ON v.`Video ID` = c.`Video ID`
""")

# Visualizar para checar
join_videos_comments_opt.show(5)

# Plano de execução otimizado
print("Plano de execução - join otimizado:")
join_videos_comments_opt.explain()

+-----------+--------------------+------------+--------+-----+--------------------+---------+
|   Video ID|               Title|Published At| Keyword|Likes|             Comment|Sentiment|
+-----------+--------------------+------------+--------+-----+--------------------+---------+
|-8TnsjDRXUE|19 Year-Old Start...|  2022-08-22|business| 3232|Amazing interview...|        2|
|-8TnsjDRXUE|19 Year-Old Start...|  2022-08-22|business| 3232|Amazing interview...|        2|
|-8TnsjDRXUE|19 Year-Old Start...|  2022-08-22|business| 3232|Amazing interview...|        2|
|-8TnsjDRXUE|19 Year-Old Start...|  2022-08-22|business| 3232|Amazing interview...|        2|
|-8TnsjDRXUE|19 Year-Old Start...|  2022-08-22|business| 3232|Amazing interview...|        2|
+-----------+--------------------+------------+--------+-----+--------------------+---------+
only showing top 5 rows

Plano de execução - join otimizado:
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [Video ID#34, Title#35, Pu

In [60]:
#8 Salva o join otimizado no formato parquet
join_videos_comments_opt.write.mode('overwrite').parquet('join-videos-comments-otimizado')

In [62]:
join_videos_comments_opt.show() # Use the correct variable name join_videos_comments_opt

+-----------+--------------------+------------+--------+-----+--------------------+---------+
|   Video ID|               Title|Published At| Keyword|Likes|             Comment|Sentiment|
+-----------+--------------------+------------+--------+-----+--------------------+---------+
|-8TnsjDRXUE|19 Year-Old Start...|  2022-08-22|business| 3232|Amazing interview...|        2|
|-8TnsjDRXUE|19 Year-Old Start...|  2022-08-22|business| 3232|Amazing interview...|        2|
|-8TnsjDRXUE|19 Year-Old Start...|  2022-08-22|business| 3232|Amazing interview...|        2|
|-8TnsjDRXUE|19 Year-Old Start...|  2022-08-22|business| 3232|Amazing interview...|        2|
|-8TnsjDRXUE|19 Year-Old Start...|  2022-08-22|business| 3232|Amazing interview...|        2|
|-8TnsjDRXUE|19 Year-Old Start...|  2022-08-22|business| 3232|Amazing interview...|        2|
|-8TnsjDRXUE|19 Year-Old Start...|  2022-08-22|business| 3232|Amazing interview...|        2|
|-8TnsjDRXUE|19 Year-Old Start...|  2022-08-22|business| 323

In [63]:
spark.stop()